In [1]:
import os
import mne
from collections import defaultdict

In [2]:
def prepare_eeg_data(base_path):
    """Cria um dicionário com caminhos completos como chaves"""
    file_index = {}
    
    for category in ['epilepsy', 'no_epilepsy']:
        category_path = os.path.join(base_path, category)
        if not os.path.exists(category_path):
            print(f"Diretório não encontrado: {category_path}")
            continue

        for root, _, files in os.walk(category_path):
            for file in files:
                if file.lower().endswith('.edf'):
                    full_path = os.path.normpath(os.path.join(root, file))
                    file_index[full_path] = {
                        'file_path': full_path,
                        'category': category
                    }
    
    return file_index

def process_by_path(file_index, target_path):
    """Processa um arquivo específico usando o caminho completo"""
    target_path = os.path.normpath(target_path)
    
    if target_path not in file_index:
        print(f"Arquivo não encontrado no índice: {target_path}")
        return None
    
    try:
        raw = mne.io.read_raw_edf(target_path, preload=False)
        print(f"\n{'-'*60}")
        print(f"Arquivo processado: {target_path}")
        print(f"Categoria: {file_index[target_path]['category']}")
        print(f"Número de canais: {len(raw.ch_names)}")
        print(f"Frequência de amostragem: {raw.info['sfreq']} Hz")
        return raw
    except Exception as e:
        print(f"Erro ao carregar {target_path}: {str(e)}")
        return None

In [9]:
def analyze_channels_by_class(file_index, output_file='channel_report_by_class.txt'):
    """Analisa canais separando por classe e gera relatório detalhado"""
    # Estruturas para armazenar dados por classe
    class_data = {
        'epilepsy': {
            'channel_sets': [],
            'channel_counter': defaultdict(int),
            'file_count': 0
        },
        'no_epilepsy': {
            'channel_sets': [],
            'channel_counter': defaultdict(int),
            'file_count': 0
        }
    }

    with open(output_file, 'w') as report:
        # Cabeçalho do relatório
        report.write("Relatório de Canais por Classe\n")
        report.write("="*50 + "\n\n")

        # Processar cada arquivo
        for path, info in file_index.items():
            category = info['category']
            class_data[category]['file_count'] += 1

            try:
                raw = mne.io.read_raw_edf(path, preload=False)
                channels = set(raw.ch_names)
                class_data[category]['channel_sets'].append(channels)

                # Atualizar contagem de canais para a categoria
                for ch in channels:
                    class_data[category]['channel_counter'][ch] += 1

            except Exception as e:
                report.write(f"Erro ao processar {path}: {str(e)}\n\n")
                continue

        # Função auxiliar para escrever seções por classe
        def write_class_section(category, data):
            report.write(f"\n{'#'*40}\n")
            report.write(f"Análise para classe: {category.upper()}\n")
            report.write(f"Total de arquivos: {data['file_count']}\n")

            if data['file_count'] > 0:
                common_channels = set.intersection(*data['channel_sets']) if data['channel_sets'] else set()
                
                report.write("\nCanais comuns a TODOS os registros:\n")
                if common_channels:
                    report.write("- " + "\n- ".join(sorted(common_channels)) + "\n")
                else:
                    report.write("Nenhum canal comum a todos os registros\n")

                report.write("\nFrequência de canais (%):\n")
                total_files = data['file_count']
                for ch, count in sorted(data['channel_counter'].items(), key=lambda x: x[1], reverse=True):
                    report.write(f"- {ch}: {count/total_files:.1%} ({count} arquivos)\n")

        # Escrever seções para cada classe
        for category in ['epilepsy', 'no_epilepsy']:
            write_class_section(category, class_data[category])

        # Resumo comparativo
        report.write("\n\n" + "="*50 + "\n")
        report.write("Resumo Comparativo:\n")
        report.write(f"Total Epilepsia: {class_data['epilepsy']['file_count']} arquivos\n")
        report.write(f"Total Não-Epilepsia: {class_data['no_epilepsy']['file_count']} arquivos\n")
        
        # Canais únicos por classe
        epilepsy_channels = set(class_data['epilepsy']['channel_counter'].keys())
        no_epilepsy_channels = set(class_data['no_epilepsy']['channel_counter'].keys())
        
        report.write("\nCanais exclusivos da classe Epilepsia:\n")
        unique_epilepsy = epilepsy_channels - no_epilepsy_channels
        report.write(
            ("- " + "\n- ".join(sorted(unique_epilepsy)) + "\n") 
            if unique_epilepsy 
            else "Nenhum\n"
        )

        report.write("\n\nCanais exclusivos da classe Não-Epilepsia:\n")
        unique_no_epilepsy = no_epilepsy_channels - epilepsy_channels
        report.write(
            ("- " + "\n- ".join(sorted(unique_no_epilepsy)) + "\n" 
            if unique_no_epilepsy 
            else "Nenhum\n"
        ))

    print(f"Relatório gerado em: {os.path.abspath(output_file)}")

In [10]:
# Preparar os dados
base_path = '/Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf'
file_index = prepare_eeg_data(base_path)

# Exemplo de uso 1: Processar arquivo específico
caminho_alvo = '/Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf/epilepsy/03_tcp_ar_a/076/00007656/s010_2011_03_13/00007656_s010_t000.edf'  # Cole seu caminho completo aqui
raw_data = process_by_path(file_index, caminho_alvo)


Extracting EDF parameters from /Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf/epilepsy/03_tcp_ar_a/076/00007656/s010_2011_03_13/00007656_s010_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

------------------------------------------------------------
Arquivo processado: /Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf/epilepsy/03_tcp_ar_a/076/00007656/s010_2011_03_13/00007656_s010_t000.edf
Categoria: epilepsy
Número de canais: 34
Frequência de amostragem: 256.0 Hz


In [11]:
# Gerar relatório separado por classes
analyze_channels_by_class(file_index)

Extracting EDF parameters from /Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf/epilepsy/03_tcp_ar_a/076/00007656/s010_2011_03_13/00007656_s010_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf/epilepsy/03_tcp_ar_a/076/00007656/s010_2011_03_13/00007656_s010_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf/epilepsy/03_tcp_ar_a/076/00007656/s010_2011_03_13/00007656_s010_t003.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/analuiza/Documents/codes/templedata/tuh_eeg_epilepsy/v1.0.0/edf/epilepsy/03_tcp_ar_a/076/00007656/s010_2011_03_13/00007656_s010_t002.edf...
EDF file detected
Setting channel info 